In [1]:
import os, random, csv, datetime, json
import pandas as pd
import numpy as np
import argparse
import time
import pickle
from tqdm import tqdm
from sklearn.model_selection import train_test_split

train_dir="../data/train/"
test_dir="../data/test/"

link_dir="../data/"

%pwd

'/home/cseadmin/undergrad1/dz/GISCUP2021/data_process'

In [2]:
def gen_feats(task, head, path, nl, link_attr, link_time_attr, driver_attr):
########################
###head features
########################
    head['speed'] = head['distance']/head['simple_eta']
    head['speed_avg'] = head['slice_id'].map(head.groupby('slice_id')['speed'].mean())
    head['speed_std'] = head['slice_id'].map(head.groupby('slice_id')['speed'].std())
    head['speed_min'] = head['slice_id'].map(head.groupby('slice_id')['speed'].min())
    head['speed_max'] = head['slice_id'].map(head.groupby('slice_id')['speed'].max())
    head['volume'] = head['slice_id'].map(head.groupby('slice_id').size())
    head['old_driver'] = head['driver_id'].map(head.groupby('driver_id').size())

    head['driver_avg'] = head['driver_id'].map(driver_attr['avg'])
    head['driver_std'] = head['driver_id'].map(driver_attr['std'])
    head['driver_min'] = head['driver_id'].map(driver_attr['min'])
    head['driver_max'] = head['driver_id'].map(driver_attr['max'])
    head['driver_cnt'] = head['driver_id'].map(driver_attr['count'])
    
    head.set_index('order_id', inplace=True)
    
    path['link_eta'] = path['link_time']*path['link_ratio']
    path['cum_time'] = (path.groupby('order_id')['link_eta'].cumsum()/300).astype('int')
    link = path[path['cross_flag']==0]
    cross = path[path['cross_flag']==1]
########################
###link features
########################
    link = link[link['link_time']>0]
    link['link_id'] = link['link_id'].astype('int')
    link['link_type'] = link['link_id'].map(link_attr['link_label']).fillna(0).astype('int')
    link['next_link'] = link['link_id'].map(nl).fillna(0).astype('int')
    link['slice_id'] = link['order_id'].map(head['slice_id'])

    link["link_time_std"] = link["link_id"].map(link_time_attr["std"])
    link["link_time_avg"] = link["link_id"].map(link_time_attr["avg"])
    link["link_time_min"] = link["link_id"].map(link_time_attr["min"])
    link["link_time_max"] = link["link_id"].map(link_time_attr["max"])
    link["link_time_count"] = link["link_id"].map(link_time_attr["count"])
    
    gl = link.groupby('order_id')
    head['link_no'] = head.index.map(gl.size())
    head['link_time_sum'] = head.index.map(gl['link_eta'].sum())
    head['link_time_avg'] = head.index.map(gl['link_time'].mean())
    head['link_time_std'] = head.index.map(gl['link_time'].std())
    head['link_time_max'] = head.index.map(gl['link_time'].max())
    head['link_time_min'] = head.index.map(gl['link_time'].min())

# -----------------------------------------------------------------------------
    head['link_time_std_sum'] = head.index.map(gl['link_time_std'].sum())
    head['link_time_std_avg'] = head.index.map(gl['link_time_std'].mean())
    head['link_time_std_std'] = head.index.map(gl['link_time_std'].std())
    head['link_time_std_min'] = head.index.map(gl['link_time_std'].min())
    head['link_time_std_max'] = head.index.map(gl['link_time_std'].max())

    head['link_time_avg_sum'] = head.index.map(gl['link_time_avg'].sum())
    head['link_time_avg_avg'] = head.index.map(gl['link_time_avg'].mean())
    head['link_time_avg_std'] = head.index.map(gl['link_time_avg'].std())
    head['link_time_avg_min'] = head.index.map(gl['link_time_avg'].min())
    head['link_time_avg_max'] = head.index.map(gl['link_time_avg'].max())

    head['link_time_max_sum'] = head.index.map(gl['link_time_max'].sum())
    head['link_time_max_avg'] = head.index.map(gl['link_time_max'].mean())
    head['link_time_max_std'] = head.index.map(gl['link_time_max'].std())
    head['link_time_max_min'] = head.index.map(gl['link_time_max'].min())
    head['link_time_max_max'] = head.index.map(gl['link_time_max'].max())

    head['link_time_min_sum'] = head.index.map(gl['link_time_min'].sum())
    head['link_time_min_avg'] = head.index.map(gl['link_time_min'].mean())
    head['link_time_min_std'] = head.index.map(gl['link_time_min'].std())
    head['link_time_min_min'] = head.index.map(gl['link_time_min'].min())
    head['link_time_min_max'] = head.index.map(gl['link_time_min'].max())

    head['link_time_count_sum'] = head.index.map(gl['link_time_count'].sum())
    head['link_time_count_avg'] = head.index.map(gl['link_time_count'].mean())
    head['link_time_count_std'] = head.index.map(gl['link_time_count'].std())
    head['link_time_count_min'] = head.index.map(gl['link_time_count'].min())
    head['link_time_count_max'] = head.index.map(gl['link_time_count'].max())
# -----------------------------------------------------------------------------
    
    head['time_delay_max'] = head.index.map(gl['cum_time'].max())
    head['time_delay_avg'] = head.index.map(gl['cum_time'].mean())
    head['time_delay_std'] = head.index.map(gl['cum_time'].std())
    
    gl = link.groupby(['order_id','link_current_status'])['link_time']
    head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('current_no_'))
    head = head.join(gl.sum().unstack(level=-1,fill_value=0).add_prefix('current_time_sum_'))
    head = head.join(gl.mean().unstack(level=-1,fill_value=0).add_prefix('current_time_avg_'))
    head = head.join(gl.std().unstack(level=-1,fill_value=0).add_prefix('current_time_std_'))
    head = head.join(gl.max().unstack(level=-1,fill_value=0).add_prefix('current_time_max_'))
    head = head.join(gl.min().unstack(level=-1,fill_value=0).add_prefix('current_time_min_'))

    gl = link.groupby(['order_id','next_link'])['link_time']
    head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('next_no_'))
    head = head.join(gl.sum().unstack(level=-1,fill_value=0).add_prefix('next_time_sum_'))
    head = head.join(gl.mean().unstack(level=-1,fill_value=0).add_prefix('next_time_avg_'))
    head = head.join(gl.std().unstack(level=-1,fill_value=0).add_prefix('next_time_std_'))
    head = head.join(gl.max().unstack(level=-1,fill_value=0).add_prefix('next_time_max_'))
    head = head.join(gl.min().unstack(level=-1,fill_value=0).add_prefix('next_time_min_'))

    gl = link.groupby(['order_id','link_type'])['link_time']
    head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('type_no_'))
    head = head.join(gl.sum().unstack(level=-1,fill_value=0).add_prefix('type_time_sum_'))
    head = head.join(gl.mean().unstack(level=-1,fill_value=0).add_prefix('type_time_avg_'))
    head = head.join(gl.std().unstack(level=-1,fill_value=0).add_prefix('type_time_std_'))
    head = head.join(gl.max().unstack(level=-1,fill_value=0).add_prefix('type_time_max_'))
    head = head.join(gl.min().unstack(level=-1,fill_value=0).add_prefix('type_time_min_'))
    
#     gl = link.groupby(['order_id','link_arrival_status'])['link_time']
#     head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('arrival_no_'))
#     head = head.join(gl.sum().unstack(level=-1,fill_value=0).add_prefix('arrival_time_sum_'))
#     head = head.join(gl.mean().unstack(level=-1,fill_value=0).add_prefix('arrival_time_avg_'))
#     head = head.join(gl.std().unstack(level=-1,fill_value=0).add_prefix('arrival_time_std_'))
#     head = head.join(gl.max().unstack(level=-1,fill_value=0).add_prefix('arrival_time_max_'))
#     head = head.join(gl.min().unstack(level=-1,fill_value=0).add_prefix('arrival_time_min_'))

########################
###cross features
########################
    gc = cross.groupby('order_id') 
    head['cross_no'] = head.index.map(gc.size())
    head['cross_sum'] = head.index.map(gc['link_time'].sum())
    head['cross_avg'] = head.index.map(gc['link_time'].mean())
    head['cross_std'] = head.index.map(gc['link_time'].std())
    head['cross_max'] = head.index.map(gc['link_time'].max())
    head['cross_min'] = head.index.map(gc['link_time'].min())
#     head['cross_ratio'] = head['cross_sum']/head['simple_eta']

    head = head.fillna(0).reset_index().set_index('slice_id')
    gl = link.groupby(['slice_id','link_current_status'])
    head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('link_status_no_'))
    head = head.fillna(0).reset_index()

    if(task=='train'):
        head['date'] = f[:8]
    return head

In [3]:
network = pd.read_table(link_dir+'nextlinks.txt',sep=' ',names=['link_id','next_link'])
network['next_link_no'] = network['next_link'].apply(lambda z: 4 if len(list(z.split(',')))>4 else len(list(z.split(','))))
nl = network.set_index('link_id')['next_link_no']

In [8]:
link_attr = pd.read_csv(link_dir+'link_attr.csv')
link_attr.set_index('Unnamed: 0',inplace=True)

link_time_attr = pd.read_csv(link_dir+'link_time_attr.csv')
link_time_attr.set_index('Unnamed: 0',inplace=True)

driver_attr = pd.read_csv(link_dir+'driver_attr.csv')
driver_attr.set_index('Unnamed: 0',inplace=True)

link_time_attr.head(10)

,std,avg,min,max,count
Unnamed: 0,,,,,
157298,3.142503,5.023350,0.3210,14.4000,137.0
511622,1.827190,3.757110,0.0000,16.5600,144.0
524542,1.259789,2.886754,0.0000,12.9600,155.0
97293,1.929949,4.649475,2.0020,20.8800,161.0
273845,1.726321,4.192694,2.5297,18.7200,160.0
556273,2.016241,4.887015,2.6341,21.6000,162.0
73842,1.432121,3.350948,0.2437,14.4000,165.0
101675,1.427683,3.406463,2.0000,14.4000,166.0
325444,0.093270,0.699338,0.0000,1.7053,5337.0


In [18]:
TRAIN_FILES = ['202008'+str(i).zfill(2)+'.csv' for i in range(1,32)]

for i,f in tqdm(enumerate(TRAIN_FILES)):    
    head = pd.read_csv(train_dir+'train_head/'+f)
    path = pd.read_csv(train_dir+'train_path/'+f)

    if(head.shape[0]==0):
        continue
    if i==0:
        train = gen_feats('train', head, path, nl, link_attr, link_time_attr, driver_attr)
    else:
        train = train.append(gen_feats('train', head, path, nl, link_attr, link_time_attr, driver_attr))

31it [30:01, 58.12s/it]


In [19]:
head = pd.read_csv(test_dir+'test_head.csv')
path = pd.read_csv(test_dir+'test_path.csv')

test = gen_feats('test', head, path, nl, link_attr, link_time_attr, driver_attr)
test['date'] = '20200901'

生成并保存 train set 和 test set

In [ ]:
%%time

train['date'] = pd.to_datetime(train['date'].astype('str'))
test['date'] = pd.to_datetime(test['date'])

for df in [train, test]:
    df['day_bias'] = (df['date']-pd.Timestamp(2020,8,1)).dt.days
    df['weekday'] = df['date'].dt.weekday<5
    df['cross_no'] = df['cross_no']*(df['cross_sum']>0)
    df['link_length'] = df['distance']/df['link_no']
    df['log_distance'] = np.log(df['distance'])
    df['log_simple_eta'] = np.log(df['simple_eta'])
    
    for i in range(5):
        df['current_no_ratio_'+str(i)] = df['current_no_'+str(i)]/df['link_no']
        df['current_time_ratio_'+str(i)] = df['current_time_sum_'+str(i)]/(df['simple_eta']-df['cross_sum'])
        df['next_no_ratio_'+str(i)] = df['next_no_'+str(i)]/df['link_no']
        df['next_time_ratio_'+str(i)] = df['next_time_sum_'+str(i)]/(df['simple_eta']-df['cross_sum'])
    for i in range(4):
        df['type_no_ratio_'+str(i)] = df['type_no_'+str(i)]/df['link_no']
        df['type_time_ratio_'+str(i)] = df['type_time_sum_'+str(i)]/(df['simple_eta']-df['cross_sum'])

In [ ]:
%%time

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
    
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)
print(train.info())
print(test.info())

In [ ]:
train.to_pickle(train_dir+'train.pkl')
test.to_pickle(test_dir+'test.pkl')